IMPORTING ULTRALYTICS LIBRARY


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
HOME=os.getcwd()
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.1/78.2 GB disk)


In [5]:
from ultralytics import YOLO
from IPython.display import display, Image

IMPORTING DATASET FROM ROBOFLOW(WITH YAML FILE WHICH CONTAINS NO OF CLASSES AND TRAIN VAL LOCATIONS

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="dQgIFmcXM86FAuTYdF7v")
project = rf.workspace("matthew-cgj0a").project("laser_2")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.117, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to laser_2-1 in yolov8:: 100%|██████████| 766/766 [00:00<00:00, 3100.29it/s]


In [ ]:
%cd /content/drive/MyDrive/laser_2-1

/content/drive/MyDrive/laser_2-1


TRAINING CUSTOM YOLOV8 MODEL

In [ ]:
model = YOLO('yolov8n.yaml')#nano
model.train(data='data.yaml', epochs=50)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

TESTING THE TRAINED MODEL ON A SAMPLE VIDEO FOR VERIFICATION

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/content/laser_2-1/runs/detect/train/weights/best.pt')

# Open the video file
video_path = "/content/drive/MyDrive/pexels-cottonbro-studio-8923305-4096x2160-25fps.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        #cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 352x640 1 target, 65.1ms
Speed: 2.5ms preprocess, 65.1ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 10.0ms
Speed: 3.1ms preprocess, 10.0ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 25.6ms
Speed: 3.5ms preprocess, 25.6ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 2 targets, 17.6ms
Speed: 4.2ms preprocess, 17.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 9.3ms
Speed: 3.1ms preprocess, 9.3ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 9.4ms
Speed: 2.7ms preprocess, 9.4ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 2 targets, 10.8ms
Speed: 4.6ms preprocess, 10.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 9.2ms
Speed: 3.1ms preprocess, 9.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 6

DETECTION ON A SAMPLE VIDEO AND SAVING THE VIDEO IN CORRESPODING OUTPUT PATH

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/content/drive/MyDrive/laser_2-1/runs/detect/train/weights/best.pt')

# Open the video file
video_path = "/content/drive/MyDrive/pexels-cottonbro-studio-8923305-4096x2160-25fps.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create a video writer object
output_path = '/content/drive/MyDrive/annotated_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Write the annotated frame to the output video
        output_video.write(annotated_frame)

        # Display the annotated frame
        #cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object, video writer, and close the display window
cap.release()
output_video.release()
cv2.destroyAllWindows()



0: 352x640 1 target, 20.0ms
Speed: 6.5ms preprocess, 20.0ms inference, 8.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 13.7ms
Speed: 11.9ms preprocess, 13.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 23.6ms
Speed: 6.5ms preprocess, 23.6ms inference, 9.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 2 targets, 9.5ms
Speed: 4.9ms preprocess, 9.5ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 11.4ms
Speed: 2.9ms preprocess, 11.4ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 9.4ms
Speed: 3.4ms preprocess, 9.4ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 2 targets, 10.6ms
Speed: 3.4ms preprocess, 10.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 target, 9.4ms
Speed: 4.6ms preprocess, 9.4ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 

In [ ]:
#SPARSE optical flow
'''import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time


lk_params = dict(winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners = 20,
                    qualityLevel = 0.3,
                    minDistance = 10,
                    blockSize = 7 )


trajectory_len = 40
detect_interval = 5
trajectories = []
frame_idx = 0


cap = cv2.VideoCapture('/content/drive/MyDrive/Untitled.mp4')


while True:

    # start time to calculate FPS
    start = time.time()

    suc,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = frame.copy()

    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)
        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
        d = abs(p0-p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_trajectories = []

        # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            trajectory.append((x, y))
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            new_trajectories.append(trajectory)
            # Newest detected point
            cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)

        trajectories = new_trajectories

        # Draw all the trajectories
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)


    # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255

        # Lastest point in latest trajectory
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv2.circle(mask, (x, y), 5, 0, -1)

        # Detect the good features to track
        p = cv2.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
        if p is not None:
            # If good features can be tracked - add that to the trajectories
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])


    frame_idx += 1
    prev_gray = frame_gray

    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)

    # Show Results
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    #cv2_imshow(img)
    #cv2_imshow(mask)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()'''

In [21]:
'''import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners = 1,
                      qualityLevel=0.3,
                      minDistance=1,
                      blockSize=7)

trajectory_len = 20
detect_interval = 1
trajectories = []
frame_idx = 0

cap = cv2.VideoCapture('/content/drive/MyDrive/Untitled.mp4')

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec for the output video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter('output7.mp4', fourcc, fps, (width, height))

while True:
    # Start time to calculate FPS
    start = time.time()

    suc, frame = cap.read()
    if not suc:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = frame.copy()

    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)
        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
        d = abs(p0 - p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_trajectories = []

        # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            trajectory.append((x, y))
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            new_trajectories.append(trajectory)
            # Newest detected point
            cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)

        trajectories = new_trajectories

        # Draw all the trajectories
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

    # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255

        # Lastest point in latest trajectory
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv2.circle(mask, (x, y), 5, 0, -1)

        # Detect the good features to track
        p = cv2.goodFeaturesToTrack(frame_gray, mask=mask, **feature_params)
        if p is not None:
            # If good features can be tracked - add that to the trajectories
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])

    frame_idx += 1
    prev_gray = frame_gray

    # End time
    end = time.time()
    # Calculate the FPS for the current frame detection
    fps = 1 / (end - start)

    # Show results
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    output.write(img)  # Write the frame with trajectories to the output video

    #cv2.imshow('Output', img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
output.release()
cv2.destroyAllWindows()'''


In [23]:
#using houghcircles

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

circle_params = dict(dp=1,
                     minDist=20,
                     param1=50,
                     param2=30,
                     minRadius=20,
                     maxRadius=30)

trajectory_len = 20
detect_interval = 1
trajectories = []
frame_idx = 0

cap = cv2.VideoCapture('/content/drive/MyDrive/Untitled.mp4')

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec for the output video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter('output9.mp4', fourcc, fps, (width, height))

while True:
    # Start time to calculate FPS
    start = time.time()

    suc, frame = cap.read()
    if not suc:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = frame.copy()

    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)
        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
        d = abs(p0 - p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_trajectories = []

        # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            trajectory.append((x, y))
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            new_trajectories.append(trajectory)
            # Newest detected point
            cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)

        trajectories = new_trajectories

        # Draw all the trajectories
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

    # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255

        # Detect circular objects
        circles = cv2.HoughCircles(frame_gray, cv2.HOUGH_GRADIENT, **circle_params)
        if circles is not None:
            circles = np.round(circles[0, :]).astype(int)
            for (x, y, r) in circles:
                trajectories.append([(x, y)])

    frame_idx += 1
    prev_gray = frame_gray

    # End time
    end = time.time()
    # Calculate the FPS for the current frame detection
    fps = 1 / (end - start)

    # Show results
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    output.write(img)  # Write the frame with trajectories to the output video

    #cv2.imshow('Output', img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
output.release()
cv2.destroyAllWindows()
